In [6]:
import h5py
import numpy as np
import pandas as pd
import os, sys
from astropy import units as u
import random

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import rc
plt.style.use('classic')
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('figure', facecolor='w')
rc('xtick', labelsize=20)
rc('ytick', labelsize=20)

from dtaidistance import dtw, clustering
from dtaidistance import dtw_visualisation as dtwvis

from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, linkage

sys.path.append('/astro/users/jbirky/projects/tess_binaries')
os.environ['TESS_DATA'] = '/data/epyc/projects2/tess'

import tess_binaries as tb

In [7]:
tess_xm = pd.read_csv(tb.cat_dir + '/asassn_tess_xm.csv.gz')
psamp = tess_xm[~np.isnan(tess_xm['period'])]
ref = psamp[psamp['period'] < 28]

In [8]:
bad_ids = np.array([368, 212, 276, 275, 483, 1103, 381, 1105, 957, 127, 1116, 13, 961, 1054, 840, 544, 136, 596, 17, 1122, 19, 231, 752, 967, 969, 1127, 547, 1142, 549, 554, 603, \
           1154, 759, 662, 605, 413, 32, 856, 860, 861, 977, 766, 317, 1192, 984, 563, 869, 609, 680, 162, 47, 872, 990, 171, 684, 173, 11209, 1213, 1215, 512, 175, 514, \
           1071, 997, 262, 441, 1002, 777, 78, 1003, 79, 520, 180, 778, 701, 444, 1011, 1015, 780, 709, 712, 190, 191, 714, 577, 784, 345, 902, 905, 529, 908, 1269, 1080, \
           618, 619, 1018, 348, 1270, 1272, 622, 1274, 1277, 716, 792, 349, 578, 1028, 1090, 794, 626, 450, 1091, 923, 534, 628, 799, 802, 805, 806, 807, 808, 110, 934, 937, \
           1299, 581, 942, 943, 539, 814, 949, 952, 1047, 1050, 737, 479, 1051, 0, 285, 8, 845, 749, 141, 655, 756, 501, 604, 857, 665, 562, 65, 775, 440, 1242, 613, 81, \
           1267, 616, 353, 922, 533, 451, 1296, 936, 1040, 939, 118, 582, 473, 561, 657, 999, 1073, 1244, 516, 178, 1007, 1265, 1016, 1075, 1019, 198, 1021, 1023, 1037, 1039, \
           580, 119, 1044, 1045, 480, 747, 966, 652, 1132, 560, 868, 1234 , 572, 614, 87, 267, 1087, 1026, 1032, 200, 816, 821, 53,   88,  140,  189,  243,  321,  435,  542,  \
           566,  588,  825, 1049, 1066, 1094, 1172, 1199, 18,  223,  552,  647,  675,  919, 944, 73])
good_ids = np.array(list(set(np.arange(0,len(ref))) - set(bad_ids)))

In [4]:
bad_per = np.array([229807000, 302965929,  80427866, 311086887, 165124012, 179050471,
       245865916,  27629711, 178547903,  25669623, 465075473, 462146751,
       129181676, 411956826, 159454564, 193565852, 91369561, 54018297, 131793466, 47380518, 169249901, 359830202, 437091565,
      270038761])
other = [35664974, 140460192, 32792896, 234222724]

In [5]:
np.array([np.where(ref['tic_id'] == b)[0][0] for b in other])

array([220, 237, 487, 510])

In [10]:
df = {'tic_id':[], 'type':[], 'period':[], 'sector':[]}
for i in good_ids:
    df['tic_id'].append(list(ref['tic_id'])[i])
    df['type'].append(list(ref['Type'])[i])
    df['period'].append(list(ref['period'])[i])
    df['sector'].append(list(ref['sector'])[i])
train = pd.DataFrame(data=df)
len(train)

1175

In [11]:
tsteps = 100
tarr = np.arange(0,tsteps,1)
pharr = np.linspace(0,1,tsteps)

table = train
# demo = random.choices(np.arange(0,len(train)), k=N)
sample = {'tic_id':[], 'type':[], 'period':[], 'flux':[]}

for i in range(len(table)):
    try:
        data_full = tb.readSourceFiles(table['tic_id'][i], sector=table['sector'][i])[0]     
        flux = tb.preprocessData(data_full['pdcsap_flux'][i], table['period'][i], tsteps=tsteps, scale=False)
#         data      = data_full.fold(period=table['period'][i]*u.day) 
#         bin_flux  = tb.binData(data, tsteps)

#         flux = np.roll(np.array(bin_flux), tsteps-np.argmin(bin_flux))

#         data = np.vstack([tarr, flux]).T
#         scaler = MinMaxScaler()
#         scaler.fit(data)
#         flux = scaler.transform(data).T[1]

        sample['flux'].append(flux)
        sample['tic_id'].append(table['tic_id'][i])
        sample['period'].append(table['period'][i])
        sample['type'].append(table['type'][i])
    except:
        print('BAD ID', i, table['tic_id'][i])

/astro/users/jbirky/.local/lib/python3.6/site-packages/astropy/timeseries/io/kepler.py:77: UserWarning: Ignoring 815 rows with NaN times
  warnings.warn('Ignoring {0} rows with NaN times'.format(np.sum(nans)))


Stacking 1 light curve(s).
BAD ID 0 30317282
Stacking 1 light curve(s).
BAD ID 1 30531417
Stacking 1 light curve(s).
BAD ID 2 31653503
Stacking 1 light curve(s).
BAD ID 3 31655792
Stacking 1 light curve(s).
BAD ID 4 32000625
Stacking 1 light curve(s).
BAD ID 5 32200310
Stacking 1 light curve(s).
BAD ID 6 33862174
Stacking 1 light curve(s).
BAD ID 7 41232835
Stacking 1 light curve(s).
BAD ID 8 51991595
Stacking 1 light curve(s).
BAD ID 9 53842685
Stacking 1 light curve(s).
BAD ID 10 53851254
Stacking 1 light curve(s).
BAD ID 11 71274771
Stacking 1 light curve(s).
BAD ID 12 71335815
Stacking 1 light curve(s).
BAD ID 13 89018905
Stacking 1 light curve(s).
BAD ID 14 99465976
Stacking 1 light curve(s).
BAD ID 15 115015715
Stacking 1 light curve(s).
BAD ID 16 117515123
Stacking 1 light curve(s).
BAD ID 17 117544915
Stacking 1 light curve(s).
BAD ID 18 121420805
Stacking 1 light curve(s).
BAD ID 19 126602778
Stacking 1 light curve(s).
BAD ID 20 126637765
Stacking 1 light curve(s).
BAD ID 21 1

/astro/users/jbirky/.local/lib/python3.6/site-packages/astropy/timeseries/io/kepler.py:77: UserWarning: Ignoring 1038 rows with NaN times
  warnings.warn('Ignoring {0} rows with NaN times'.format(np.sum(nans)))


Stacking 1 light curve(s).
BAD ID 104 8963901
Stacking 1 light curve(s).
BAD ID 105 8989778
Stacking 1 light curve(s).
BAD ID 106 30631330
Stacking 1 light curve(s).
BAD ID 107 33864387
Stacking 1 light curve(s).
BAD ID 108 41865879
Stacking 1 light curve(s).
BAD ID 109 41896812
Stacking 1 light curve(s).
BAD ID 110 43398254
Stacking 1 light curve(s).
BAD ID 111 49735506
Stacking 1 light curve(s).
BAD ID 112 69838619
Stacking 1 light curve(s).
BAD ID 113 69949079
Stacking 1 light curve(s).
BAD ID 114 70742090
Stacking 1 light curve(s).
BAD ID 115 70742156
Stacking 1 light curve(s).
BAD ID 116 70744201
Stacking 1 light curve(s).
BAD ID 117 70761941
Stacking 1 light curve(s).
BAD ID 118 80427281
Stacking 1 light curve(s).
BAD ID 119 89543393
Stacking 1 light curve(s).
BAD ID 120 100029948
Stacking 1 light curve(s).
BAD ID 121 101963752
Stacking 1 light curve(s).
BAD ID 122 114819903
Stacking 1 light curve(s).
BAD ID 123 117549840
Stacking 1 light curve(s).
BAD ID 124 117874959
Stacking 1

/astro/users/jbirky/.local/lib/python3.6/site-packages/astropy/timeseries/io/kepler.py:77: UserWarning: Ignoring 280 rows with NaN times
  warnings.warn('Ignoring {0} rows with NaN times'.format(np.sum(nans)))


...

Stacking 1 light curve(s).
BAD ID 856 7082633
Stacking 1 light curve(s).
BAD ID 857 49541462
Stacking 1 light curve(s).
BAD ID 858 60436937
Stacking 1 light curve(s).
BAD ID 859 62124646
Stacking 1 light curve(s).
BAD ID 860 79662102
Stacking 1 light curve(s).
BAD ID 861 90723908
Stacking 1 light curve(s).
BAD ID 862 90911675
Stacking 1 light curve(s).
BAD ID 863 91853165
Stacking 1 light curve(s).
BAD ID 864 101433435
Stacking 1 light curve(s).
BAD ID 865 112401201
Stacking 1 light curve(s).
BAD ID 866 118053599
Stacking 1 light curve(s).
BAD ID 867 118195419
Stacking 1 light curve(s).
BAD ID 868 120334451
Stacking 1 light curve(s).
BAD ID 869 161083755
Stacking 1 light curve(s).
BAD ID 870 182894226
Stacking 1 light curve(s).
BAD ID 871 194025647
Stacking 1 light curve(s).
BAD ID 872 253742057
Stacking 1 light curve(s).
BAD ID 873 261283885
Stacking 1 light curve(s).
BAD ID 874 273305007
Stacking 1 light curve(s).
BAD ID 875 316582319
Stacking 1 light curve(s).
BAD ID 876 318649597
S

/astro/users/jbirky/.local/lib/python3.6/site-packages/astropy/timeseries/io/kepler.py:77: UserWarning: Ignoring 683 rows with NaN times
  warnings.warn('Ignoring {0} rows with NaN times'.format(np.sum(nans)))


Stacking 1 light curve(s).
BAD ID 891 10129567
Stacking 1 light curve(s).
BAD ID 892 11119600
Stacking 1 light curve(s).
BAD ID 893 12915124
Stacking 1 light curve(s).
BAD ID 894 14323558
Stacking 1 light curve(s).
BAD ID 895 14617089
Stacking 1 light curve(s).
BAD ID 896 15795549
Stacking 1 light curve(s).
BAD ID 897 26656583
Stacking 1 light curve(s).
BAD ID 898 27307817
Stacking 1 light curve(s).
BAD ID 899 27315670
Stacking 1 light curve(s).
BAD ID 900 27843942
Stacking 1 light curve(s).
BAD ID 901 28359831
Stacking 1 light curve(s).
BAD ID 902 40378419
Stacking 1 light curve(s).
BAD ID 903 41840710
Stacking 1 light curve(s).
BAD ID 904 41998476
Stacking 1 light curve(s).
BAD ID 905 47626069
Stacking 1 light curve(s).
BAD ID 906 48446609
Stacking 1 light curve(s).
BAD ID 907 48499902
Stacking 1 light curve(s).
BAD ID 908 63006634
Stacking 1 light curve(s).
BAD ID 909 63074282
Stacking 1 light curve(s).
BAD ID 910 63205800
Stacking 1 light curve(s).
BAD ID 911 63820329
Stacking 1 li

/astro/users/jbirky/.local/lib/python3.6/site-packages/astropy/timeseries/io/kepler.py:77: UserWarning: Ignoring 776 rows with NaN times
  warnings.warn('Ignoring {0} rows with NaN times'.format(np.sum(nans)))


Stacking 1 light curve(s).
BAD ID 1075 11113164
Stacking 1 light curve(s).
BAD ID 1076 11205157
Stacking 1 light curve(s).
BAD ID 1077 11480287
Stacking 1 light curve(s).
BAD ID 1078 11480757
Stacking 1 light curve(s).
BAD ID 1079 11704852
Stacking 1 light curve(s).
BAD ID 1080 11918748
Stacking 1 light curve(s).
BAD ID 1081 13976604
Stacking 1 light curve(s).
BAD ID 1082 14206846
Stacking 1 light curve(s).
BAD ID 1083 14844585
Stacking 1 light curve(s).
BAD ID 1084 16493058
Stacking 1 light curve(s).
BAD ID 1085 16617827
Stacking 1 light curve(s).
BAD ID 1086 17451194
Stacking 1 light curve(s).
BAD ID 1087 40367312
Stacking 1 light curve(s).
BAD ID 1088 55778901
Stacking 1 light curve(s).
BAD ID 1089 63328020
Stacking 1 light curve(s).
BAD ID 1090 63853144
Stacking 1 light curve(s).
BAD ID 1091 67343900
Stacking 1 light curve(s).
BAD ID 1092 84070123
Stacking 1 light curve(s).
BAD ID 1093 92776364
Stacking 1 light curve(s).
BAD ID 1094 117638854
Stacking 1 light curve(s).
BAD ID 1095 

In [25]:
df = pd.DataFrame(data=sample)
fname = f'{tb.cat_dir}/asassn_tess_inspected.hdf5'

In [26]:
with h5py.File(fname, 'w') as f:
    f.create_dataset('tic_id', data=df['tic_id'])
    f.create_dataset('period', data=df['period'])
    f.create_dataset('type', data=df['type'], dtype=h5py.special_dtype(vlen=str))
    f.create_dataset('flux', data=df['flux'], dtype=h5py.special_dtype(vlen=np.dtype('float64')))
f.close()

In [ ]:
ff = h5py.File(f'{tb.cat_dir}/asassn_tess_inspected.hdf5', mode="r")

dd = {}
for key in list(ff):
    dd[key] = ff[key].value
    
ff.close()

In [24]:
len(sample)

1004

In [17]:
bad_ids = np.array(['91369561','152223725','270038761','340219000','41865879','308451526','41865879','306736831','314826925','54018297','452357628','31281820','170882537','219203188','50745582','102202982','110793065','131793466','265206385','47380518','169249901','188209486','29850366','359830202','396004353','437091565','41169654','67271271','229807000','302965929','311086887','144598282','179050471','245865916','27629711','178547903','25669623','465075473','462146751','129181676','411956826','159454564','193565852'])

In [23]:
sample = sample[~sample['tic_id'].isin(bad_ids)]

In [20]:
ff = h5py.File(f'{tb.cat_dir}/asassn_tess_inspected_0.hdf5', mode="r")

df = {}
for key in list(ff):
    if key == 'type':
        df[key] = np.array(ff[key].value, dtype='str')
    else:
        df[key] = ff[key].value
    
ff.close()
sample = pd.DataFrame(data=df)

/epyc/opt/anaconda/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
